## 国家数据分析实践

In [1]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 3.6rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}
.rendered_html img[src*="#thumbnailauto"], .rendered_html svg[src*="#thumbnail"] {
    max-width: auto;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-reshape {
    width: 550px;
    height: 205px;
    background: url('../20春_pandas_CheatSheet.svg') -205px -110px;
    float: left;
}
.break {
                   page-break-after: right; 
                   width:100%;
                   clear:both;
}
</style>

### step01:准备数据

In [50]:
import pandas as pds

In [31]:
# 读取数据
df = pd.read_csv("fsnd_zb_meta.tsv", encoding="utf8", sep="\t")
df.head()

,code,cname,dotcount,exp,ifshowcode,memo,name,nodesort,sortcode,tag,unit
0,A010101,地级区划数,0,指地级行政单位即介于省级和县级之间的一级地方行政区域的个数，包括地区、自治州、行政区和盟。,False,NaN,地级区划数,1,2,NaN,个
1,A010102,地级市数,0,市是省、自治区内人口较集中，政治、经济、文化等方面较重要的城市。市人民政府为一级地方行政组织...,False,NaN,地级市数,1,3,NaN,个
2,A010103,县级区划数,0,县级行政单位指中国地方二级行政区域，是地方政权的基础。县级行政单位包括县、自治县、旗、自治旗...,False,NaN,县级区划数,1,4,NaN,个
3,A010104,市辖区数,0,市辖区（简称区）城市基层政权组织的行政区域。直辖市和较大的市多将市区范围划分为若干区，设立区...,False,NaN,市辖区数,1,5,NaN,个
4,A010105,县级市数,0,县级市是中国大陆行政区划名称，行政地位与县相同的县级行政区,False,NaN,县级市数,1,6,NaN,个


In [32]:
# 国家数据库分省数据准备
df_raw = pd.read_csv("fsnd_zb_data.tsv", encoding="utf8", sep="\t", \
                    keep_default_na=False, na_values = 'na_rep',
                    index_col = [0, 1, 2])
df_m =  pd.read_csv("fsnd_zb_meta.tsv", encoding="utf8", sep="\t", \
                    keep_default_na=False,)
df_r =  pd.read_csv("reg_treeId_level2.tsv", encoding="utf8", sep="\t", \
                    keep_default_na=False, )
display(df_raw)

data
zb      reg    sj             
A010101 110000 2018        NaN
               2017        NaN
               2016        NaN
               2015        NaN
               2014        NaN
               2013        NaN
               2012        NaN
               2011        NaN
               2010        NaN
               2009        NaN
        120000 2018        NaN
               2017        NaN
               2016        NaN
               2015        NaN
               2014        NaN
               2013        NaN
               2012        NaN
               2011        NaN
               2010        NaN
               2009        NaN
        130000 2018        NaN
               2017  11.000000
               2016  11.000000
               2015  11.000000
               2014  11.000000
               2013  11.000000
               2012  11.000000
               2011  11.000000
               2010  11.000000
               2009  11.000000
...                        ...
A0S0B05 630000 2018        NaN
               2017        NaN
               2016        NaN
               2015  21.646300
               2014  16.455900
               2013  11.984900
               2012   8.854700
               2011        NaN
               2010        NaN
               2009        NaN
        640000 2018        NaN
               2017        NaN
               2016        NaN
               2015  19.288500
               2014  16.148400
               2013  12.473900
               2012   8.530635
               2011        NaN
               2010        NaN
               2009        NaN
        650000 2018        NaN
               2017        NaN
               2016        NaN
               2015  52.169500
               2014  42.592900
               2013  33.856600
               2012  24.914044
               2011        NaN
               2010        NaN
               2009        NaN

[908300 rows x 1 columns]

In [33]:
# 查看数组有多少
df_raw.shape

(908300, 1)

### step02:创建字典

In [44]:
# 创建指标字典
指标字典 = df_m.set_index("code")['cname'].to_dict() # 取cname转化为字典
指标字典

{'A010101': '地级区划数',
 'A010102': '地级市数',
 'A010103': '县级区划数',
 'A010104': '市辖区数',
 'A010105': '县级市数',
 'A010106': '县数',
 'A010107': '自治县数',
 'A010108': '乡镇级区划数',
 'A010109': '镇数',
 'A01010A': '乡数',
 'A01010B': '街道办事处',
 'A010201': '三次产业法人单位数',
 'A010202': '分机构类型法人单位数',
 'A010203': '分行业法人单位数',
 'A010301': '按控股情况分企业法人单位数',
 'A010302': '按登记注册类型分企业法人单位数',
 'A020101': '地区生产总值',
 'A020102': '第一产业增加值',
 'A020103': '第二产业增加值',
 'A020104': '第三产业增加值',
 'A020105': '农林牧渔业增加值',
 'A020106': '工业增加值',
 'A020107': '建筑业增加值',
 'A020108': '批发和零售业增加值',
 'A020109': '批发和零售贸易餐饮业增加值',
 'A02010A': '交通运输、仓储和邮政业增加值',
 'A02010B': '交通运输、仓储和邮电通信业增加值',
 'A02010C': '住宿和餐饮业增加值',
 'A02010D': '金融业增加值',
 'A02010E': '房地产业增加值',
 'A02010F': '其他行业增加值',
 'A02010G': '人均地区生产总值',
 'A020201': '地区生产总值指数(上年=100)',
 'A020202': '第一产业增加值指数(上年=100)',
 'A020203': '第二产业增加值指数(上年=100)',
 'A020204': '第三产业增加值指数(上年=100)',
 'A020301': '收入法生产总值',
 'A020302': '劳动者报酬',
 'A020303': '生产税净额',
 'A020304': '固定资产折旧',
 'A020305': '营业盈余',
 'A020401': '支出法生

In [35]:
df_m

,code,cname,dotcount,exp,ifshowcode,memo,name,nodesort,sortcode,tag,unit
0,A010101,地级区划数,0,指地级行政单位即介于省级和县级之间的一级地方行政区域的个数，包括地区、自治州、行政区和盟。,False,,地级区划数,1,2,,个
1,A010102,地级市数,0,市是省、自治区内人口较集中，政治、经济、文化等方面较重要的城市。市人民政府为一级地方行政组织...,False,,地级市数,1,3,,个
2,A010103,县级区划数,0,县级行政单位指中国地方二级行政区域，是地方政权的基础。县级行政单位包括县、自治县、旗、自治旗...,False,,县级区划数,1,4,,个
3,A010104,市辖区数,0,市辖区（简称区）城市基层政权组织的行政区域。直辖市和较大的市多将市区范围划分为若干区，设立区...,False,,市辖区数,1,5,,个
4,A010105,县级市数,0,县级市是中国大陆行政区划名称，行政地位与县相同的县级行政区,False,,县级市数,1,6,,个
5,A010106,县数,0,县级行政区是中国地方一级行政区域。按中国现行行政区域体制，县可属于省、自治区、直辖市，可属于...,False,,县数,1,7,,个
6,A010107,自治县数,0,自治县是相当于县的民族自治地方。,False,,自治县数,1,8,,个
7,A010108,乡镇级区划数,0,县级以下的基层行政区域──乡和镇，是地方三级行政单位。,False,,乡镇级区划数,1,9,,个
8,A010109,镇数,0,指报告期末不设区的市、市辖区、县（自治县、旗、自治旗、特区、林区）在辖区内实际设有的镇人民政...,False,,镇数,1,10,,个
9,A01010A,乡数,0,指报告期末不设区的市、市辖区、县（自治县、旗、自治旗、特区、林区）在辖区内实际设有的乡人民政...,False,,乡数,1,11,,个


In [36]:
# 创建地区字典
地区字典 = df_r.set_index("id")['name'].to_dict() #取id为key，name为value
地区字典

{110000: '北京市',
 120000: '天津市',
 130000: '河北省',
 140000: '山西省',
 150000: '内蒙古自治区',
 210000: '辽宁省',
 220000: '吉林省',
 230000: '黑龙江省',
 310000: '上海市',
 320000: '江苏省',
 330000: '浙江省',
 340000: '安徽省',
 350000: '福建省',
 360000: '江西省',
 370000: '山东省',
 410000: '河南省',
 420000: '湖北省',
 430000: '湖南省',
 440000: '广东省',
 450000: '广西壮族自治区',
 460000: '海南省',
 500000: '重庆市',
 510000: '四川省',
 520000: '贵州省',
 530000: '云南省',
 540000: '西藏自治区',
 610000: '陕西省',
 620000: '甘肃省',
 630000: '青海省',
 640000: '宁夏回族自治区',
 650000: '新疆维吾尔自治区'}

In [37]:
df_r.head()

,i,dbcode,exp,id,isParent,name,open,pid,wd
0,0,fsnd,,110000,True,北京市,False,100001,reg
1,1,fsnd,,120000,True,天津市,False,100001,reg
2,2,fsnd,,130000,True,河北省,False,100001,reg
3,3,fsnd,,140000,True,山西省,False,100001,reg
4,4,fsnd,,150000,True,内蒙古自治区,False,100001,reg


### step03:使用字典进行数据框转换

In [45]:
# 指标字典==>zb指标
df = df_raw.reset_index().set_index("zb").rename(index=指标字典)
df
# zb变为索引

,reg,sj,data
zb,,,
地级区划数,110000,2018,NaN
地级区划数,110000,2017,NaN
地级区划数,110000,2016,NaN
地级区划数,110000,2015,NaN
地级区划数,110000,2014,NaN
地级区划数,110000,2013,NaN
地级区划数,110000,2012,NaN
地级区划数,110000,2011,NaN
地级区划数,110000,2010,NaN


In [46]:
# 地区字典==>reg地区
# [地区字典[x] for x in set(df.reset_index()['reg'].to_list())]
df = df.reset_index().set_index("reg").rename(index=地区字典)
df

,zb,sj,data
reg,,,
北京市,地级区划数,2018,NaN
北京市,地级区划数,2017,NaN
北京市,地级区划数,2016,NaN
北京市,地级区划数,2015,NaN
北京市,地级区划数,2014,NaN
北京市,地级区划数,2013,NaN
北京市,地级区划数,2012,NaN
北京市,地级区划数,2011,NaN
北京市,地级区划数,2010,NaN


In [48]:
# 重命名columns
df_zh = df.reset_index().rename(columns={"zb":"指标","reg":"地区",\
                                         "sj":"年","data":"数据",})
df_zh

,地区,指标,年,数据
0,北京市,地级区划数,2018,NaN
1,北京市,地级区划数,2017,NaN
2,北京市,地级区划数,2016,NaN
3,北京市,地级区划数,2015,NaN
4,北京市,地级区划数,2014,NaN
5,北京市,地级区划数,2013,NaN
6,北京市,地级区划数,2012,NaN
7,北京市,地级区划数,2011,NaN
8,北京市,地级区划数,2010,NaN
9,北京市,地级区划数,2009,NaN


### step04:切数据：实现快速查找

In [79]:
# 例：查找国有城镇单位就业人员工资总额数据
df_zh.set_index('指标').loc["国有城镇单位就业人员工资总额","年":"数据"]

,年,数据
指标,,
国有城镇单位就业人员工资总额,2018,NaN
国有城镇单位就业人员工资总额,2017,2663.84409
国有城镇单位就业人员工资总额,2016,2350.74965
国有城镇单位就业人员工资总额,2015,2106.60123
国有城镇单位就业人员工资总额,2014,1932.38647
国有城镇单位就业人员工资总额,2013,1771.35664
国有城镇单位就业人员工资总额,2012,1624.84200
国有城镇单位就业人员工资总额,2011,1460.24719
国有城镇单位就业人员工资总额,2010,1264.30000


In [49]:
# 按问题意识去切数据
dslice = df_zh [ df_zh.指标.str.contains("城镇单位就业人员")] # 筛选出“城镇单位就业人员”
dslice

,地区,指标,年,数据
19530,北京市,城镇单位就业人员,2018,NaN
19531,北京市,城镇单位就业人员,2017,812.8589
19532,北京市,城镇单位就业人员,2016,791.5197
19533,北京市,城镇单位就业人员,2015,777.3448
19534,北京市,城镇单位就业人员,2014,755.8601
19535,北京市,城镇单位就业人员,2013,742.2565
19536,北京市,城镇单位就业人员,2012,717.3717
19537,北京市,城镇单位就业人员,2011,685.9018
19538,北京市,城镇单位就业人员,2010,646.6000
19539,北京市,城镇单位就业人员,2009,619.3478


### step05：思考“分”进合击之“分”的可能性

In [51]:
# 思考“分”进合击之“分”的可能性
指标分的可能性 = dslice.指标.unique() # unique:以数组形式（numpy.ndarray）返回列的所有唯一值（特征的所有唯一值）
指标分的可能性
# 结构是？
# “人员、工资总额、平均工资、平均实际工资”的全行业总和与各行业数据

array(['城镇单位就业人员', '农林牧渔业城镇单位就业人员', '采矿业城镇单位就业人员', '制造业城镇单位就业人员',
       '电力、燃气及水的生产和供应业城镇单位就业人员', '建筑业城镇单位就业人员', '交通运输、仓储及邮电通信业城镇单位就业人员',
       '信息传输、计算机服务和软件业城镇单位就业人员', '批发和零售业城镇单位就业人员', '住宿和餐饮业城镇单位就业人员',
       '金融业城镇单位就业人员', '房地产业城镇单位就业人员', '租赁和商务服务业城镇单位就业人员',
       '科学研究、技术服务和地质勘查业城镇单位就业人员', '水利、环境和公共设施管理业城镇单位就业人员',
       '居民服务和其他服务业城镇单位就业人员', '教育业城镇单位就业人员', '卫生、社会保障和社会福利业城镇单位就业人员',
       '文化、体育和娱乐业城镇单位就业人员', '公共管理和社会组织城镇单位就业人员', '城镇单位就业人员工资总额',
       '国有城镇单位就业人员工资总额', '其他城镇单位就业人员工资总额', '城镇单位就业人员工资总额指数(上年=100)',
       '国有城镇单位就业人员工资总额指数(上年=100)', '其他城镇单位就业人员工资总额指数(上年=100)',
       '城镇单位就业人员平均工资', '城镇单位就业人员平均货币工资指数(上年=100)',
       '国有城镇单位就业人员平均货币工资指数(上年=100)', '其他城镇单位就业人员平均货币工资指数(上年=100)',
       '城镇单位就业人员平均实际工资指数(上年=100)', '国有城镇单位就业人员平均实际工资指数(上年=100)',
       '其他城镇单位就业人员平均实际工资指数(上年=100)', '农、林、牧、渔业城镇单位就业人员工资总额',
       '采矿业城镇单位就业人员工资总额', '制造业城镇单位就业人员工资总额', '电力、燃气及水的生产和供应业城镇单位就业人员工资总额',
       '建筑业城镇单位就业人员工资总额', '交通运输、仓储和邮政业城镇单位就业人员工资总额',
       '信息传输、计算机服务和软件业城镇单位就业人员工资总额', 

In [52]:
指标分的可能性 = [ x.split("城镇单位就业") for x in dslice.指标.unique()]
指标分的可能性

[['', '人员'],
 ['农林牧渔业', '人员'],
 ['采矿业', '人员'],
 ['制造业', '人员'],
 ['电力、燃气及水的生产和供应业', '人员'],
 ['建筑业', '人员'],
 ['交通运输、仓储及邮电通信业', '人员'],
 ['信息传输、计算机服务和软件业', '人员'],
 ['批发和零售业', '人员'],
 ['住宿和餐饮业', '人员'],
 ['金融业', '人员'],
 ['房地产业', '人员'],
 ['租赁和商务服务业', '人员'],
 ['科学研究、技术服务和地质勘查业', '人员'],
 ['水利、环境和公共设施管理业', '人员'],
 ['居民服务和其他服务业', '人员'],
 ['教育业', '人员'],
 ['卫生、社会保障和社会福利业', '人员'],
 ['文化、体育和娱乐业', '人员'],
 ['公共管理和社会组织', '人员'],
 ['', '人员工资总额'],
 ['国有', '人员工资总额'],
 ['其他', '人员工资总额'],
 ['', '人员工资总额指数(上年=100)'],
 ['国有', '人员工资总额指数(上年=100)'],
 ['其他', '人员工资总额指数(上年=100)'],
 ['', '人员平均工资'],
 ['', '人员平均货币工资指数(上年=100)'],
 ['国有', '人员平均货币工资指数(上年=100)'],
 ['其他', '人员平均货币工资指数(上年=100)'],
 ['', '人员平均实际工资指数(上年=100)'],
 ['国有', '人员平均实际工资指数(上年=100)'],
 ['其他', '人员平均实际工资指数(上年=100)'],
 ['农、林、牧、渔业', '人员工资总额'],
 ['采矿业', '人员工资总额'],
 ['制造业', '人员工资总额'],
 ['电力、燃气及水的生产和供应业', '人员工资总额'],
 ['建筑业', '人员工资总额'],
 ['交通运输、仓储和邮政业', '人员工资总额'],
 ['信息传输、计算机服务和软件业', '人员工资总额'],
 ['批发和零售业', '人员工资总额'],
 ['住宿和餐饮业', '人员工资总额'],
 ['金融业', '人员工资总额'],
 ['房地产

In [54]:
# 只取“人员”，“人员平均工资”
指标分的可能性_取 = [ [x,y] for (x,y) in 指标分的可能性 if (y=='人员平均工资' or y=='人员') and x!='']
指标分的可能性_取

[['农林牧渔业', '人员'],
 ['采矿业', '人员'],
 ['制造业', '人员'],
 ['电力、燃气及水的生产和供应业', '人员'],
 ['建筑业', '人员'],
 ['交通运输、仓储及邮电通信业', '人员'],
 ['信息传输、计算机服务和软件业', '人员'],
 ['批发和零售业', '人员'],
 ['住宿和餐饮业', '人员'],
 ['金融业', '人员'],
 ['房地产业', '人员'],
 ['租赁和商务服务业', '人员'],
 ['科学研究、技术服务和地质勘查业', '人员'],
 ['水利、环境和公共设施管理业', '人员'],
 ['居民服务和其他服务业', '人员'],
 ['教育业', '人员'],
 ['卫生、社会保障和社会福利业', '人员'],
 ['文化、体育和娱乐业', '人员'],
 ['公共管理和社会组织', '人员'],
 ['农、林、牧、渔业', '人员平均工资'],
 ['采矿业', '人员平均工资'],
 ['制造业', '人员平均工资'],
 ['电力、燃气及水的生产和供应业', '人员平均工资'],
 ['建筑业', '人员平均工资'],
 ['交通运输、仓储和邮政业', '人员平均工资'],
 ['信息传输、计算机服务和软件业', '人员平均工资'],
 ['批发和零售业', '人员平均工资'],
 ['住宿和餐饮业', '人员平均工资'],
 ['金融业', '人员平均工资'],
 ['房地产业', '人员平均工资'],
 ['租赁和商务服务业', '人员平均工资'],
 ['科学研究、技术服务和地质勘查业', '人员平均工资'],
 ['水利、环境和公共设施管理业', '人员平均工资'],
 ['居民服务和其他服务业', '人员平均工资'],
 ['教育', '人员平均工资'],
 ['卫生、社会保障和社会福利业', '人员平均工资'],
 ['文化、体育和娱乐业', '人员平均工资'],
 ['公共管理和社会组织', '人员平均工资']]

In [55]:
# 进一步切片
指标分的可能性_取_all = ["城镇单位就业".join(x) for x in 指标分的可能性_取]
指标分的可能性_取_all

['农林牧渔业城镇单位就业人员',
 '采矿业城镇单位就业人员',
 '制造业城镇单位就业人员',
 '电力、燃气及水的生产和供应业城镇单位就业人员',
 '建筑业城镇单位就业人员',
 '交通运输、仓储及邮电通信业城镇单位就业人员',
 '信息传输、计算机服务和软件业城镇单位就业人员',
 '批发和零售业城镇单位就业人员',
 '住宿和餐饮业城镇单位就业人员',
 '金融业城镇单位就业人员',
 '房地产业城镇单位就业人员',
 '租赁和商务服务业城镇单位就业人员',
 '科学研究、技术服务和地质勘查业城镇单位就业人员',
 '水利、环境和公共设施管理业城镇单位就业人员',
 '居民服务和其他服务业城镇单位就业人员',
 '教育业城镇单位就业人员',
 '卫生、社会保障和社会福利业城镇单位就业人员',
 '文化、体育和娱乐业城镇单位就业人员',
 '公共管理和社会组织城镇单位就业人员',
 '农、林、牧、渔业城镇单位就业人员平均工资',
 '采矿业城镇单位就业人员平均工资',
 '制造业城镇单位就业人员平均工资',
 '电力、燃气及水的生产和供应业城镇单位就业人员平均工资',
 '建筑业城镇单位就业人员平均工资',
 '交通运输、仓储和邮政业城镇单位就业人员平均工资',
 '信息传输、计算机服务和软件业城镇单位就业人员平均工资',
 '批发和零售业城镇单位就业人员平均工资',
 '住宿和餐饮业城镇单位就业人员平均工资',
 '金融业城镇单位就业人员平均工资',
 '房地产业城镇单位就业人员平均工资',
 '租赁和商务服务业城镇单位就业人员平均工资',
 '科学研究、技术服务和地质勘查业城镇单位就业人员平均工资',
 '水利、环境和公共设施管理业城镇单位就业人员平均工资',
 '居民服务和其他服务业城镇单位就业人员平均工资',
 '教育城镇单位就业人员平均工资',
 '卫生、社会保障和社会福利业城镇单位就业人员平均工资',
 '文化、体育和娱乐业城镇单位就业人员平均工资',
 '公共管理和社会组织城镇单位就业人员平均工资']

In [56]:
df_就业切片 = df_zh.set_index("指标").loc[指标分的可能性_取_all].reset_index() #把“指标”换成“指标分的可能性_取_all”
df_就业切片

,指标,地区,年,数据
0,农林牧渔业城镇单位就业人员,北京市,2018,NaN
1,农林牧渔业城镇单位就业人员,北京市,2017,3.4116
2,农林牧渔业城镇单位就业人员,北京市,2016,3.6867
3,农林牧渔业城镇单位就业人员,北京市,2015,3.8949
4,农林牧渔业城镇单位就业人员,北京市,2014,3.2331
5,农林牧渔业城镇单位就业人员,北京市,2013,3.1346
6,农林牧渔业城镇单位就业人员,北京市,2012,2.5446
7,农林牧渔业城镇单位就业人员,北京市,2011,2.3632
8,农林牧渔业城镇单位就业人员,北京市,2010,3.2000
9,农林牧渔业城镇单位就业人员,北京市,2009,3.2282


In [57]:
# 查看参数的意义
df_就业切片.describe(include="all")

,指标,地区,年,数据
count,11780,11780,11780.000000,10591.000000
unique,38,31,NaN,NaN
top,科学研究、技术服务和地质勘查业城镇单位就业人员平均工资,新疆维吾尔自治区,NaN,NaN
freq,310,380,NaN,NaN
mean,NaN,NaN,2013.500000,25995.610415
std,NaN,NaN,2.872403,31837.310363
min,NaN,NaN,2009.000000,0.024900
25%,NaN,NaN,2011.000000,11.323850
50%,NaN,NaN,2013.500000,11478.000000
75%,NaN,NaN,2016.000000,46218.500000


In [58]:
df_就业切片

,指标,地区,年,数据
0,农林牧渔业城镇单位就业人员,北京市,2018,NaN
1,农林牧渔业城镇单位就业人员,北京市,2017,3.4116
2,农林牧渔业城镇单位就业人员,北京市,2016,3.6867
3,农林牧渔业城镇单位就业人员,北京市,2015,3.8949
4,农林牧渔业城镇单位就业人员,北京市,2014,3.2331
5,农林牧渔业城镇单位就业人员,北京市,2013,3.1346
6,农林牧渔业城镇单位就业人员,北京市,2012,2.5446
7,农林牧渔业城镇单位就业人员,北京市,2011,2.3632
8,农林牧渔业城镇单位就业人员,北京市,2010,3.2000
9,农林牧渔业城镇单位就业人员,北京市,2009,3.2282


### step06: 分进合击出报表

### 分组数据

In [59]:
df_就业切片.groupby(['指标']).agg({"数据":["min","mean","max"]})

数据                           
                                    min          mean          max
指标                                                                
交通运输、仓储及邮电通信业城镇单位就业人员            0.5997     24.556320      85.3999
交通运输、仓储和邮政业城镇单位就业人员平均工资      25098.0000  56241.240143  116763.0000
住宿和餐饮业城镇单位就业人员                   0.3000      8.331891      39.3419
住宿和餐饮业城镇单位就业人员平均工资           13455.0000  31198.731183   61095.0000
信息传输、计算机服务和软件业城镇单位就业人员           0.2235      9.204604      77.4400
信息传输、计算机服务和软件业城镇单位就业人员平均工资   22186.0000  72507.896057  212063.0000
公共管理和社会组织城镇单位就业人员                7.8169     50.301842     116.1762
公共管理和社会组织城镇单位就业人员平均工资        25275.0000  55019.594982  128855.0000
农、林、牧、渔业城镇单位就业人员平均工资          8832.0000  30661.906810   74975.0000
农林牧渔业城镇单位就业人员                    0.3089     10.091744      93.9292
制造业城镇单位就业人员                      0.6612    145.443355    1020.2491
制造业城镇单位就业人员平均工资              21508.0000  45198.247312  106835.0000
卫生、社会保障和社会福利业城镇单位就业人员            1.3830     24.421601      65.8223
卫生、社会保障和社会福利业城镇单位就业人员平均工资    22302.0000  59073.053763  169191.0000
居民服务和其他服务业城镇单位就业人员               0.0557      2.277055      11.0436
居民服务和其他服务业城镇单位就业人员平均工资       16022.0000  35929.476703   67013.0000
建筑业城镇单位就业人员                      0.5018     72.355329     450.1977
建筑业城镇单位就业人员平均工资              16423.0000  40809.896057   99718.0000
房地产业城镇单位就业人员                     0.0249     10.849628      66.3147
房地产业城镇单位就业人员平均工资             15938.0000  45288.530466  120379.0000
批发和零售业城镇单位就业人员                   0.4833     24.356891     102.7835
批发和零售业城镇单位就业人员平均工资           16073.0000  44120.035842  139627.0000
教育业城镇单位就业人员                      3.9384     53.813387     127.4604
教育城镇单位就业人员平均工资               26175.0000  57414.487455  143215.0000
文化、体育和娱乐业城镇单位就业人员                0.5999      4.581411      19.0189
文化、体育和娱乐业城镇单位就业人员平均工资        22377.0000  54176.698925  150810.0000
水利、环境和公共设施管理业城镇单位就业人员            0.1636      8.022701      18.2523
水利、环境和公共设施管理业城镇单位就业人员平均工资    15831.0000  36995.433692   95341.0000
电力、燃气及水的生产和供应业城镇单位就业人员           0.7362     11.707497      32.1821
电力、燃气及水的生产和供应业城镇单位就业人员平均工资   29419.0000  67744.182796  174252.0000
科学研究、技术服务和地质勘查业城镇单位就业人员          0.6790     11.614666      71.2481
科学研究、技术服务和地质勘查业城镇单位就业人员平均工资  26211.0000  66216.186380  176383.0000
租赁和商务服务业城镇单位就业人员                 0.1000     12.672472      88.2695
租赁和商务服务业城镇单位就业人员平均工资         16691.0000  43966.784946  156621.0000
采矿业城镇单位就业人员                      0.0523     18.220504     103.0136
采矿业城镇单位就业人员平均工资              22732.0000  56747.756272  144454.0000
金融业城镇单位就业人员                      0.7717     17.982375      54.4498
金融业城镇单位就业人员平均工资              30627.0000  86977.516129  253637.0000

### 多层次数据

In [60]:
df_就业切片['行业'] = [x.split("城镇单位就业")[0] for x in df_就业切片.指标]
df_就业切片['行业指标'] = [x.split("城镇单位就业")[1] for x in df_就业切片.指标]
df_就业切片

,指标,地区,年,数据,行业,行业指标
0,农林牧渔业城镇单位就业人员,北京市,2018,NaN,农林牧渔业,人员
1,农林牧渔业城镇单位就业人员,北京市,2017,3.4116,农林牧渔业,人员
2,农林牧渔业城镇单位就业人员,北京市,2016,3.6867,农林牧渔业,人员
3,农林牧渔业城镇单位就业人员,北京市,2015,3.8949,农林牧渔业,人员
4,农林牧渔业城镇单位就业人员,北京市,2014,3.2331,农林牧渔业,人员
5,农林牧渔业城镇单位就业人员,北京市,2013,3.1346,农林牧渔业,人员
6,农林牧渔业城镇单位就业人员,北京市,2012,2.5446,农林牧渔业,人员
7,农林牧渔业城镇单位就业人员,北京市,2011,2.3632,农林牧渔业,人员
8,农林牧渔业城镇单位就业人员,北京市,2010,3.2000,农林牧渔业,人员
9,农林牧渔业城镇单位就业人员,北京市,2009,3.2282,农林牧渔业,人员


In [61]:
df_就业切片.groupby(['地区','行业','行业指标']).agg({"数据":["min","mean","max"]})

数据                            
                                     min           mean          max
地区   行业              行业指标                                           
上海市  交通运输、仓储及邮电通信业   人员          35.6329      45.014167      51.4541
     交通运输、仓储和邮政业     人员平均工资   49847.0000   81817.888889  116763.0000
     住宿和餐饮业          人员          10.8191      19.851000      25.5494
                     人员平均工资   29564.0000   45158.777778   60153.0000
     信息传输、计算机服务和软件业  人员           6.5150      17.706767      30.7312
                     人员平均工资  101367.0000  153913.666667  212063.0000
     公共管理和社会组织       人员          18.2518      19.983900      21.1526
                     人员平均工资   65919.0000   92678.222222  118964.0000
     农、林、牧、渔业        人员平均工资   38093.0000   54100.666667   69903.0000
     农林牧渔业           人员           1.2254       1.876556       2.7646
     制造业             人员         140.7073     183.501878     218.7271
                     人员平均工资   46672.0000   73948.888889  105733.0000
     卫生、社会保障和社会福利业   人员          16.6032      18.130889      19.1078
                     人员平均工资   66826.0000  104244.444444  138074.0000
     居民服务和其他服务业      人员           2.3978       4.984733       7.4547
                     人员平均工资   30875.0000   51385.888889   67013.0000
     建筑业             人员          11.0132      31.301400      48.8552
                     人员平均工资   51894.0000   71907.222222   96580.0000
     房地产业            人员          11.2000      20.145967      26.7351
                     人员平均工资   45903.0000   70926.333333   98474.0000
     批发和零售业          人员          25.4025      60.077933      80.0561
                     人员平均工资   52553.0000   97942.111111  139627.0000
     教育              人员平均工资   61763.0000   89319.111111  111090.0000
     教育业             人员          26.1000      28.563867      30.8006
     文化、体育和娱乐业       人员           4.5675       5.459844       6.4103
                     人员平均工资   62422.0000   97158.222222  145257.0000
     水利、环境和公共设施管理业   人员           5.9000       7.429000       8.6987
                     人员平均工资   41240.0000   60816.555556   86441.0000
     电力、燃气及水的生产和供应业  人员           4.1367       5.056733       6.1079
                     人员平均工资   83958.0000  127959.666667  174252.0000
...                                  ...            ...          ...
黑龙江省 农、林、牧、渔业        人员平均工资   11079.0000   21949.555556   30638.0000
     农林牧渔业           人员          65.5137      80.446111      93.9292
     制造业             人员          45.6676      61.749822      78.6306
                     人员平均工资   23894.0000   39142.333333   55497.0000
     卫生、社会保障和社会福利业   人员          18.5067      21.434622      23.1389
                     人员平均工资   29432.0000   45865.777778   66627.0000
     居民服务和其他服务业      人员           3.9102       4.780867       6.5000
                     人员平均工资   25848.0000   44416.222222   58569.0000
     建筑业             人员          24.7003      30.938256      37.2165
                     人员平均工资   20610.0000   32888.111111   42200.0000
     房地产业            人员           4.2734       5.677700       6.2550
                     人员平均工资   20254.0000   34830.555556   46693.0000
     批发和零售业          人员          15.9010      17.933000      19.7209
                     人员平均工资   21693.0000   37019.222222   50907.0000
     教育              人员平均工资   31400.0000   48862.888889   72656.0000
     教育业             人员          41.7976      44.863233      46.9553
     文化、体育和娱乐业       人员           3.8244       4.060133       4.5747
                     人员平均工资   28140.0000   41567.444444   58391.0000
     水利、环境和公共设施管理业   人员           9.2396      10.291267      11.1348
                     人员平均工资   16502.0000   26638.333333   36282.0000
     电力、燃气及水的生产和供应业  人员          14.3000      16.475733      18.2837
                     人员平均工资   32767.0000   51645.000000   68215.0000
     科学研究、技术服务和地质勘查业 人员          10.6923      11.466122      12.1715
                     人员平均工资   39938.0000   57595.444444   73978.0000
     租赁和商务服务业        人员           4.

### 尝试做城镇单位就业人员数量分析

In [62]:
df_就业切片.to_csv("就业切片.tsv",encoding="utf8",sep="\t")

In [85]:
# 只取“人员”
人员 = df_就业切片.query("行业指标=='人员'") #query:字符串表达式查询 

In [86]:
# 根据数据的平均值进行排序
人员.groupby(['行业','地区']).agg({"数据":["min","mean","max"]}).sort_values(by=('数据','mean'),ascending = False)

数据                       
                              min        mean        max
行业             地区                                       
制造业            广东省       443.4591  765.059322  1020.2491
               江苏省       308.2048  470.074022   612.3192
               山东省       337.2251  390.859067   437.1586
               浙江省       315.0247  344.893656   376.7476
               河南省       154.7755  271.643900   363.2567
建筑业            浙江省       155.8483  269.044322   329.4524
               江苏省        46.0405  258.315689   450.1977
制造业            福建省       213.1199  246.410300   292.3783
               上海市       140.7073  183.501878   218.7271
               湖北省       118.5083  168.209033   193.3310
               辽宁省       117.7764  152.580078   179.4581
               四川省       123.0931  152.075011   204.5781
建筑业            河南省        87.3204  147.141978   189.6092
               山东省        70.4565  139.961333   188.6478
               四川省        99.6475  134.632256   177.8360
制造业            河北省       102.5542  132.266578   150.3358
建筑业            福建省        51.9644  127.238689   176.7182
               广东省        63.1542  119.837111   162.2781
教育业            广东省       109.0172  119.648222   127.4604
               河南省       112.5064  119.216756   124.9991
制造业            湖南省        98.7284  118.569211   134.2319
建筑业            湖北省        69.8228  115.524078   144.1509
教育业            山东省       105.9537  113.795244   120.3473
制造业            江西省        68.0148  111.082378   141.7538
公共管理和社会组织      山东省       103.1073  109.838433   116.1762
               河南省       103.7958  109.116411   115.5082
制造业            安徽省        70.6086  103.244922   122.6224
公共管理和社会组织      广东省        91.3392  102.527911   114.0892
制造业            天津市        72.8962  101.207033   122.3296
               北京市        82.3796   97.864478   107.9701
...                           ...         ...        ...
农林牧渔业          浙江省         0.4262    0.874933     1.4911
文化、体育和娱乐业      宁夏回族自治区     0.7227    0.874733     1.0532
房地产业           青海省         0.6000    0.825444     1.0399
租赁和商务服务业       青海省         0.7046    0.805089     0.9076
批发和零售业         西藏自治区       0.4833    0.799000     1.1630
居民服务和其他服务业     新疆维吾尔自治区    0.4637    0.791044     1.6149
文化、体育和娱乐业      青海省         0.6649    0.783200     0.8622
采矿业            海南省         0.5008    0.770311     0.9974
交通运输、仓储及邮电通信业  西藏自治区       0.5997    0.765244     0.8972
居民服务和其他服务业     安徽省         0.4589    0.751211     1.0032
               广西壮族自治区     0.5732    0.742944     0.8174
信息传输、计算机服务和软件业 宁夏回族自治区     0.5973    0.730056     0.8431
居民服务和其他服务业     江西省         0.4836    0.725467     1.1028
农林牧渔业          西藏自治区       0.3089    0.719422     1.0928
文化、体育和娱乐业      西藏自治区       0.5999    0.668600     0.7866
农林牧渔业          天津市         0.5045    0.625733     0.8475
住宿和餐饮业         青海省         0.4771    0.605011     0.7632
               宁夏回族自治区     0.4000    0.567756     0.6772
               西藏自治区       0.3000    0.441844     0.6224
信息传输、计算机服务和软件业 西藏自治区       0.2235    0.424300     0.5241
居民服务和其他服务业     海南省         0.1752    0.373200     0.5252
采矿业            西藏自治区       0.1207    0.363089     0.5670
居民服务和其他服务业     甘肃省         0.2000    0.316256     0.3999
租赁和商务服务业       西藏自治区       0.1000    0.305556     0.4909
水利、环境和公共设施管理业  西藏自治区       0.1636    0.198433     0.2253
居民服务和其他服务业     西藏自治区       0.1490    0.198100     0.2343
               青海省         0.0557    0.182800     0.3366
房地产业           西藏自治区       0.0249    0.137333     0.2038
居民服务和其他服务业     宁夏回族自治区     0.0581    0.077040     0.0933
采矿业            上海市         0.0523    0.064189     0.1000

[589 rows x 3 columns]

In [88]:
人员平均工资.groupby(['地区']).agg({"数据":["min","mean","max"]}).sort_values(by=('数据','mean'),ascending=False)

数据                      
             min       mean        max
地区                                    
广东省       2.4535  84.940409  1020.2491
江苏省       1.0000  62.964261   612.3192
山东省       1.3845  59.890800   437.1586
浙江省       0.4262  53.427877   376.7476
河南省       1.5968  51.412716   363.2567
北京市       2.3632  38.299189   107.9701
四川省       1.0000  37.544399   204.5781
湖北省       1.2000  33.400080   193.3310
福建省       1.3135  32.270201   292.3783
河北省       1.4368  31.146511   150.3358
辽宁省       2.2826  30.751218   179.4581
湖南省       1.3259  29.344616   134.2319
上海市       0.0523  29.217498   218.7271
安徽省       0.4589  24.383643   122.6224
陕西省       1.3816  23.844242   107.9059
黑龙江省      3.2713  23.720562    93.9292
山西省       0.6249  22.464426   103.0136
江西省       0.4836  21.304642   141.7538
云南省       0.7380  20.358667    73.9954
广西壮族自治区   0.5732  19.456787    80.8122
重庆市       0.7735  19.046848   103.8066
吉林省       0.8769  15.925829    88.4034
新疆维吾尔自治区  0.4637  15.622043    67.9821
内蒙古自治区    0.7260  14.651575    47.5162
贵州省       0.9668  14.550780    56.5299
天津市       0.5045  14.110606   122.3296
甘肃省       0.2000  12.288468    46.0595
海南省       0.1752   4.904139    14.9580
宁夏回族自治区   0.0581   3.628315    12.9449
青海省       0.0557   3.169238    12.0868
西藏自治区     0.0249   1.545604    14.4254

In [89]:
_df_ = 人员.groupby(['行业']).agg({"数据":["min","mean","max"]})
display(人员)
display(_df_)
print(_df_.columns)

,指标,地区,年,数据,行业,行业指标
0,农林牧渔业城镇单位就业人员,北京市,2018,NaN,农林牧渔业,人员
1,农林牧渔业城镇单位就业人员,北京市,2017,3.4116,农林牧渔业,人员
2,农林牧渔业城镇单位就业人员,北京市,2016,3.6867,农林牧渔业,人员
3,农林牧渔业城镇单位就业人员,北京市,2015,3.8949,农林牧渔业,人员
4,农林牧渔业城镇单位就业人员,北京市,2014,3.2331,农林牧渔业,人员
5,农林牧渔业城镇单位就业人员,北京市,2013,3.1346,农林牧渔业,人员
6,农林牧渔业城镇单位就业人员,北京市,2012,2.5446,农林牧渔业,人员
7,农林牧渔业城镇单位就业人员,北京市,2011,2.3632,农林牧渔业,人员
8,农林牧渔业城镇单位就业人员,北京市,2010,3.2000,农林牧渔业,人员
9,农林牧渔业城镇单位就业人员,北京市,2009,3.2282,农林牧渔业,人员


数据                       
                    min        mean        max
行业                                            
交通运输、仓储及邮电通信业    0.5997   24.556320    85.3999
住宿和餐饮业           0.3000    8.331891    39.3419
信息传输、计算机服务和软件业   0.2235    9.204604    77.4400
公共管理和社会组织        7.8169   50.301842   116.1762
农林牧渔业            0.3089   10.091744    93.9292
制造业              0.6612  145.443355  1020.2491
卫生、社会保障和社会福利业    1.3830   24.421601    65.8223
居民服务和其他服务业       0.0557    2.277055    11.0436
建筑业              0.5018   72.355329   450.1977
房地产业             0.0249   10.849628    66.3147
批发和零售业           0.4833   24.356891   102.7835
教育业              3.9384   53.813387   127.4604
文化、体育和娱乐业        0.5999    4.581411    19.0189
水利、环境和公共设施管理业    0.1636    8.022701    18.2523
电力、燃气及水的生产和供应业   0.7362   11.707497    32.1821
科学研究、技术服务和地质勘查业  0.6790   11.614666    71.2481
租赁和商务服务业         0.1000   12.672472    88.2695
采矿业              0.0523   18.220504   103.0136
金融业              0.7717   17.982375    54.4498

MultiIndex(levels=[['数据'], ['min', 'mean', 'max']],
           codes=[[0, 0, 0], [0, 1, 2]])


In [90]:
人员.groupby(['地区']).agg({"数据":["min","mean","max"]}).columns

MultiIndex(levels=[['数据'], ['min', 'mean', 'max']],
           codes=[[0, 0, 0], [0, 1, 2]])

In [93]:
人员.groupby(['行业','地区']).agg({"数据":["min","mean","max"]})\
            .sort_values(by=('数据','mean'),ascending=False)

数据                       
                              min        mean        max
行业             地区                                       
制造业            广东省       443.4591  765.059322  1020.2491
               江苏省       308.2048  470.074022   612.3192
               山东省       337.2251  390.859067   437.1586
               浙江省       315.0247  344.893656   376.7476
               河南省       154.7755  271.643900   363.2567
建筑业            浙江省       155.8483  269.044322   329.4524
               江苏省        46.0405  258.315689   450.1977
制造业            福建省       213.1199  246.410300   292.3783
               上海市       140.7073  183.501878   218.7271
               湖北省       118.5083  168.209033   193.3310
               辽宁省       117.7764  152.580078   179.4581
               四川省       123.0931  152.075011   204.5781
建筑业            河南省        87.3204  147.141978   189.6092
               山东省        70.4565  139.961333   188.6478
               四川省        99.6475  134.632256   177.8360
制造业            河北省       102.5542  132.266578   150.3358
建筑业            福建省        51.9644  127.238689   176.7182
               广东省        63.1542  119.837111   162.2781
教育业            广东省       109.0172  119.648222   127.4604
               河南省       112.5064  119.216756   124.9991
制造业            湖南省        98.7284  118.569211   134.2319
建筑业            湖北省        69.8228  115.524078   144.1509
教育业            山东省       105.9537  113.795244   120.3473
制造业            江西省        68.0148  111.082378   141.7538
公共管理和社会组织      山东省       103.1073  109.838433   116.1762
               河南省       103.7958  109.116411   115.5082
制造业            安徽省        70.6086  103.244922   122.6224
公共管理和社会组织      广东省        91.3392  102.527911   114.0892
制造业            天津市        72.8962  101.207033   122.3296
               北京市        82.3796   97.864478   107.9701
...                           ...         ...        ...
农林牧渔业          浙江省         0.4262    0.874933     1.4911
文化、体育和娱乐业      宁夏回族自治区     0.7227    0.874733     1.0532
房地产业           青海省         0.6000    0.825444     1.0399
租赁和商务服务业       青海省         0.7046    0.805089     0.9076
批发和零售业         西藏自治区       0.4833    0.799000     1.1630
居民服务和其他服务业     新疆维吾尔自治区    0.4637    0.791044     1.6149
文化、体育和娱乐业      青海省         0.6649    0.783200     0.8622
采矿业            海南省         0.5008    0.770311     0.9974
交通运输、仓储及邮电通信业  西藏自治区       0.5997    0.765244     0.8972
居民服务和其他服务业     安徽省         0.4589    0.751211     1.0032
               广西壮族自治区     0.5732    0.742944     0.8174
信息传输、计算机服务和软件业 宁夏回族自治区     0.5973    0.730056     0.8431
居民服务和其他服务业     江西省         0.4836    0.725467     1.1028
农林牧渔业          西藏自治区       0.3089    0.719422     1.0928
文化、体育和娱乐业      西藏自治区       0.5999    0.668600     0.7866
农林牧渔业          天津市         0.5045    0.625733     0.8475
住宿和餐饮业         青海省         0.4771    0.605011     0.7632
               宁夏回族自治区     0.4000    0.567756     0.6772
               西藏自治区       0.3000    0.441844     0.6224
信息传输、计算机服务和软件业 西藏自治区       0.2235    0.424300     0.5241
居民服务和其他服务业     海南省         0.1752    0.373200     0.5252
采矿业            西藏自治区       0.1207    0.363089     0.5670
居民服务和其他服务业     甘肃省         0.2000    0.316256     0.3999
租赁和商务服务业       西藏自治区       0.1000    0.305556     0.4909
水利、环境和公共设施管理业  西藏自治区       0.1636    0.198433     0.2253
居民服务和其他服务业     西藏自治区       0.1490    0.198100     0.2343
               青海省         0.0557    0.182800     0.3366
房地产业           西藏自治区       0.0249    0.137333     0.2038
居民服务和其他服务业     宁夏回族自治区     0.0581    0.077040     0.0933
采矿业            上海市         0.0523    0.064189     0.1000

[589 rows x 3 columns]

In [73]:
df_就业切片
df_就业切片.groupby(['行业','行业指标','地区']).agg({"数据":["min","mean","max"]})

数据                            
                                       min           mean          max
行业            行业指标   地区                                               
交通运输、仓储及邮电通信业 人员     上海市           35.6329      45.014167      51.4541
                     云南省           13.4000      15.667100      17.8598
                     内蒙古自治区        16.2429      19.266189      22.7777
                     北京市           50.0662      56.857056      60.2262
                     吉林省           14.2939      15.836233      17.1587
                     四川省           22.5911      32.641667      41.3341
                     天津市           11.4261      13.687956      15.0084
                     宁夏回族自治区        2.9000       3.538267       4.0421
                     安徽省           14.7374      19.554044      24.2102
                     山东省           33.2476      42.983356      50.6405
                     山西省           20.4000      22.790011      24.5916
                     广东省           53.5351      72.138578      85.3999
                     广西壮族自治区       17.5170      19.229089      21.2419
                     新疆维吾尔自治区      10.8969      14.641267      17.4876
                     江苏省           30.6102      41.005756      49.7774
                     江西省           12.8752      17.977667      21.2442
                     河北省           24.2689      26.436611      29.1791
                     河南省           28.7767      38.154511      45.8357
                     浙江省           23.9385      29.470078      32.6864
                     海南省            4.2353       5.457289       7.0685
                     湖北省           24.3822      31.060989      35.3677
                     湖南省           21.0000      23.454667      25.1949
                     甘肃省           10.1000      11.659911      13.3029
                     福建省           16.1943      21.067956      24.5012
                     西藏自治区          0.5997       0.765244       0.8972
                     贵州省            9.1411      10.631744      12.1651
                     辽宁省           30.2000      34.420811      37.6584
                     重庆市           13.4554      21.405222      27.2819
                     陕西省           18.3423      23.817067      28.7338
                     青海省            3.2761       3.944167       4.7567
...                                    ...            ...          ...
金融业           人员平均工资 云南省        52773.0000   96004.555556  130774.0000
                     内蒙古自治区     40945.0000   64995.888889   81065.0000
                     北京市       143187.0000  204188.555556  253637.0000
                     吉林省        36473.0000   64133.333333   87154.0000
                     四川省        43556.0000   72613.111111   91529.0000
                     天津市        74068.0000  104960.777778  118448.0000
                     宁夏回族自治区    47745.0000   71067.666667   85861.0000
                     安徽省        36891.0000   62982.888889   79039.0000
                     山东省        46480.0000   75837.000000   94704.0000
                     山西省        35447.0000   63850.333333   80556.0000
                     广东省        84721.0000  115820.555556  149936.0000
                     广西壮族自治区    51315.0000   77059.333333   96818.0000
                     新疆维吾尔自治区   45502.0000   73113.111111   95374.0000
                     江苏省        60164.0000   99507.222222  126541.0000
                     江西省        33055.0000   62095.666667   84304.0000
                     河北省        36989.0000   62520.444444   77845.0000
                     河南省        38019.0000   65114.222222  103314.0000
                     浙江省        84714.0000  117956.000000  132411.0000
                     海南省        52674.0000   81065.222222  117551.0000
                     湖北省        43200.0000   71224.666667  101551.0000
                     湖南省        38337.0000   72976.222222   99320.0000
                     甘肃省        30627.0000   48098.666667   63050.0000
                     福建省        61181.0000   94084.888889  10975

In [96]:
统一字词 = {"交通运输、仓储及邮电通信业":"交通运输、仓储和邮政业",\
            "交通运输、仓储和邮政业":"交通运输丶仓储和邮政业",\
            "农、林、牧、渔业":"农林牧渔业",\
            "教育":"教育业", "教育业业":"教育业"}
df_就业切片_新 = df_就业切片.set_index('行业').rename(index=统一字词).reset_index()
df_就业切片_新.行业.unique()

array(['农林牧渔业', '采矿业', '制造业', '电力、燃气及水的生产和供应业', '建筑业', '交通运输、仓储和邮政业',
       '信息传输、计算机服务和软件业', '批发和零售业', '住宿和餐饮业', '金融业', '房地产业', '租赁和商务服务业',
       '科学研究、技术服务和地质勘查业', '水利、环境和公共设施管理业', '居民服务和其他服务业', '教育业',
       '卫生、社会保障和社会福利业', '文化、体育和娱乐业', '公共管理和社会组织', '交通运输丶仓储和邮政业'],
      dtype=object)

In [101]:
df_就业切片_新.groupby(['行业','地区','行业指标']).agg({"数据":["min","mean","max"]})

数据                            
                                    min           mean          max
行业          地区       行业指标                                          
交通运输、仓储和邮政业 上海市      人员         35.6329      45.014167      51.4541
            云南省      人员         13.4000      15.667100      17.8598
            内蒙古自治区   人员         16.2429      19.266189      22.7777
            北京市      人员         50.0662      56.857056      60.2262
            吉林省      人员         14.2939      15.836233      17.1587
            四川省      人员         22.5911      32.641667      41.3341
            天津市      人员         11.4261      13.687956      15.0084
            宁夏回族自治区  人员          2.9000       3.538267       4.0421
            安徽省      人员         14.7374      19.554044      24.2102
            山东省      人员         33.2476      42.983356      50.6405
            山西省      人员         20.4000      22.790011      24.5916
            广东省      人员         53.5351      72.138578      85.3999
            广西壮族自治区  人员         17.5170      19.229089      21.2419
            新疆维吾尔自治区 人员         10.8969      14.641267      17.4876
            江苏省      人员         30.6102      41.005756      49.7774
            江西省      人员         12.8752      17.977667      21.2442
            河北省      人员         24.2689      26.436611      29.1791
            河南省      人员         28.7767      38.154511      45.8357
            浙江省      人员         23.9385      29.470078      32.6864
            海南省      人员          4.2353       5.457289       7.0685
            湖北省      人员         24.3822      31.060989      35.3677
            湖南省      人员         21.0000      23.454667      25.1949
            甘肃省      人员         10.1000      11.659911      13.3029
            福建省      人员         16.1943      21.067956      24.5012
            西藏自治区    人员          0.5997       0.765244       0.8972
            贵州省      人员          9.1411      10.631744      12.1651
            辽宁省      人员         30.2000      34.420811      37.6584
            重庆市      人员         13.4554      21.405222      27.2819
            陕西省      人员         18.3423      23.817067      28.7338
            青海省      人员          3.2761       3.944167       4.7567
...                                 ...            ...          ...
金融业         河北省      人员         23.4191      27.347989      34.6430
                     人员平均工资  36989.0000   62520.444444   77845.0000
            河南省      人员         22.0203      24.874056      29.9962
                     人员平均工资  38019.0000   65114.222222  103314.0000
            浙江省      人员         27.9838      37.452311      48.4530
                     人员平均工资  84714.0000  117956.000000  132411.0000
            海南省      人员          1.9616       3.172489       4.7517
                     人员平均工资  52674.0000   81065.222222  117551.0000
            湖北省      人员         15.7610      17.925933      21.5371
                     人员平均工资  43200.0000   71224.666667  101551.0000
            湖南省      人员         17.2812      21.850533      26.7211
                     人员平均工资  38337.0000   72976.222222   99320.0000
            甘肃省      人员          6.6131       7.275067       8.4387
                     人员平均工资  30627.0000   48098.666667   63050.0000
            福建省      人员         12.1587      15.920767      20.8204
                     人员平均工资  61181.0000   94084.888889  109757.0000
            西藏自治区    人员          0.7717       0.936122       1.3388
                     人员平均工资  76650.0000  133585.222222  186085.0000
            贵州省      人员          5.9365       7.791189       9.1727
                     人员平均工资  53780.0000   99037.666667  141959.0000
            辽宁省      人员         20.1945      23.867033      28.3071
                     人员平均工资  45038.0000   70013.222222   85433.0000
            重庆市      人员          9.7491      12.551467      14.3960
                     人员平均工资  49871.0000   95005.666667  126739.0000
            陕西省      人员         12.8648      16.387411      22.4008
                     人员平均

### 总结（拿到一份数据应该如何用pandas分析？）

1. 读取数据（此时可以看到数据量可能很庞大）
2. 筛选某些列作为index（可以把这些列先转化为字典，再转换到数据框内）
3. 根据分析问题导向“切”数据（再次细化index）
4. 思考“分进合击”之“分”的可能性（查看当前问题导向index下还有哪些细分，并选取最终确定的index）
5. 分进合击出报表（把数据进行groupby，呈现出最终想要获得的数据报表）